# 加上两个用户行为特征


In [1]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

train_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt',sep=' ')
test_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt',sep=' ')

# 线下线上数据统一进行特征处理
test_df['is_trade'] = -1
total_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

# 1 先处理时序数据

In [2]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

total_df['date'] = total_df['context_timestamp'].apply(lambda x: extract_date(x))
total_df['day'] = le.fit_transform(total_df['date'])
total_df['weekday'] = total_df['context_timestamp'].apply(lambda x: extract_weekday(x))
total_df['hour'] = total_df['context_timestamp'].apply(lambda x: extract_hour(x))

# 2 先分离训练集, 测试集, 线上集



In [56]:
# 获取训练测试的索引
import numpy as np
starts = [0,1,2,3,4,5]
ends = [6]
train_indices = total_df[total_df['day'].isin(starts)].index.values
test_indices = total_df[total_df['day'].isin(ends)].index.values
print(train_indices.shape, test_indices.shape)

tmp_df = total_df.copy()

# 把测试索引的label提取
y_test = tmp_df.iloc[test_indices]['is_trade']

# 把需要训练和测试的数据提取, 并且强制去掉测试集的label列
tmp_df.loc[test_indices, 'is_trade'] = np.nan

# 得出训练测试必须的数据集 ,并且添加一列data_set作为标记
train_tmp = tmp_df.iloc[train_indices].copy()
train_tmp['data_set'] = 'training'
test_tmp = tmp_df.iloc[test_indices].copy()
test_tmp['data_set'] = 'testing'
raw_df = train_tmp.append(test_tmp)


(420717,) (57421,)


### 重复列特征

In [57]:
%run ../util/time_utils.py
dup_feat = ['item_id', 'item_brand_id', 'shop_id', 'user_id']
raw_df = generateColDupByDay(raw_df, dup_feat, list(range(1, 8)), verbose=False)

### 交易率特征选择

*固定smooth为10*

1. 特征是否独立提升?
2. 平滑是否不会改变正负向?

##### 正向特征

> item_city_id, shop_id, user_gender_id, item_sales_level, item_collected_level, shop_review_num_level

##### 负向特征

> item_price_level,
item_id,item_brand_id,item_pv_level,user_age_level,
user_occupation_id,user_star_level,context_page_id,
shop_review_positive_rate,shop_star_level,
shop_score_service,shop_score_description,day,hour


In [58]:
# 计算前一天的交易率set到下一天，第0天用回自己
# %run ../util/time_utils.py

# setTradeRateByDate(raw_df, ['item_city_id', 'item_id', 'item_brand_id', 'shop_id', 'user_id'])

# 计算前一天的交易率set到下一天，第0天用回自己
%run ../util/trade_info.py

# trade_rela = ['item_city_id', 'shop_id', 'user_gender_id', 'item_sales_level', 'item_collected_level', 'shop_review_num_level']
trade_rela = ['item_city_id', 'item_id', 'item_brand_id', 'shop_id', 'user_id']

# colSm = {}
# for col in trade_rela:
#     colSm[col] = [15*(mean0) , 15]
generateTradeRateByDate(raw_df, trade_rela, 7, None, verbose=False, glbSmoothing=200, glbMean0=0.05)
print(raw_df.shape)

C:\Anaconda3\lib\site-packages\pandas\core\series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


(478138, 71)


# 3 生成矩阵数据

In [59]:
train_df = raw_df[raw_df['data_set'] == 'training']
test_df = raw_df[raw_df['data_set'] == 'testing']

non_feat_columns = ['data_set', 'context_timestamp', 'instance_id', 'is_trade', 'context_id',
                   'item_property_list', 'item_category_list', 'date', 'predict_category_property'
                   ]

D = train_df[['date']]
X_train = train_df.drop(non_feat_columns, axis=1)
y_train = train_df[['is_trade']].values.ravel()
X_test = test_df.drop(non_feat_columns, axis=1)
# y_test is already exists

# X_online = test_df.drop(non_feat_columns, axis=1).values
print(D.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(420717, 1) (420717, 62) (420717,) (57421, 62) (57421,)


In [60]:
# 训练模型
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost

#import lightgbm as lgb

%run ../util/time_series_split.py

# clf = lgb.LGBMClassifier(n_jobs=20)
clf = xgboost.XGBClassifier(n_jobs=7,max_depth=5,n_estimators=100,min_child_weight=5, scale_pos_weight=1)
# clf = xgboost.XGBClassifier(n_jobs=7, max_depth=3, n_estimators=230)

m = None

# 如果移动到线上集, 则输出模型
if sum(y_test == -1) > 0:
    m = clf.fit(X_train, y_train)
    result = pd.DataFrame()
    result['instance_id'] = test_df['instance_id']
    result['predicted_score'] = pd.DataFrame(m.predict_proba(X_test))[1].values
    result.to_csv('submits/8194_a8272_b8161-Copy1.csv', sep = ' ', header=True, index = False)
else:
    # 分离a,b榜
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=6)
    m = clf.fit(X_train, y_train)
    
    val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
    val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
    print('(%s -> %s) train logloss: %.5f, test logloss: %.5f, a: %.5f, b: %.5f' % \
          (starts, ends, \
           log_loss(y_train, m.predict_proba(X_train)), \
           log_loss(y_test, m.predict_proba(X_test)),\
          val_a_loss, val_b_loss))
    
# ([0] -> [1]) train logloss: 0.09174, test logloss: 0.09354
# ([0, 1] -> [2]) train logloss: 0.05530, test logloss: 0.09213
# ([0, 1, 2] -> [3]) train logloss: 0.06654, test logloss: 0.09130
# ([0, 1, 2, 3] -> [4]) train logloss: 0.07268, test logloss: 0.08890
# ([0, 1, 2, 3, 4] -> [5]) train logloss: 0.07571, test logloss: 0.08354
# ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07691, test logloss: 0.08207
# ([0, 1, 2, 3, 4, 5] -> [6]) 调参后, train logloss: 0.07463, test logloss: 0.08194, a: 0.08272, b: 0.08161


([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07463, test logloss: 0.08194, a: 0.08272, b: 0.08161


# 4 模型分析

In [61]:
m

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=5, missing=None, n_estimators=100,
       n_jobs=7, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [62]:
# print(X_val_a.shape, y_val_a.shape, X_val_b.shape, y_val_b.shape)

# rand_states = []
# bias = 0.0
# for i in range(0, 3000):
#     X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=i)
#     # train_loss = log_loss(y_train, m.predict_proba(X_train))
    
#     val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
#     val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
#     bias = bias + val_a_loss - val_b_loss
#     if val_a_loss > 0.089:
#         rand_states.append(i)
    
#     print('第 %s 次: A: %.5f, B: %.5f, A_B_mean: %.5f, bias: %.5f' % \
#           ( i, val_a_loss, val_b_loss, (val_a_loss*0.3+val_b_loss*0.7), bias))
    
# test_loss = log_loss(y_test, m.predict_proba(X_test))
# print('test logloss: %.5f, 总的bias: %s' % (test_loss, bias))
# print(rand_states)